# train data 만들기

##### 이미지 색 데이터 추출

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

def show_origin(img):   # 원본 이미지, 크기 출력
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

    print(img_rgb.shape)

def color_hist(img):    # 원본 이미지 색 히스토그램 출력
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 그레이 이미지의 0번 채널에 대한 히스토그램 계산
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])

    # RGB 이미지의 각 채널에 대한 히스토그램 계산
    hist_b = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])

    # 히스토그램 시각화
    plt.figure(figsize=(8, 6))
    plt.title('Color Histogram')
    plt.xlabel('Color Value')
    plt.ylabel('Frequency')

    # 그레이 히스토그램 채우기
    plt.fill_between(range(256), hist.flatten(), color='k', alpha=0.4, label='Histogram')

    # RGB 채널 히스토그램 그래프 그리기
    plt.plot(hist_b, color='b', label='Blue Channel')
    plt.plot(hist_g, color='g', label='Green Channel')
    plt.plot(hist_r, color='r', label='Red Channel')

    plt.xlim([0, 256])
    plt.legend()
    plt.show()

def brightness(img_path):   # 원본 이미지 밝기 계산
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(image, (500, 750))  # (가로, 세로)

    brightness_value = round(img.mean() / 255 , 4)
    # print(f"이미지 밝기 : {brightness_value}")

    return brightness_value

def main_3_color(img):  # 원본 이미지 대표 색상 3개 rgb 값 반환
    main_3_color_list = []

    # 이미지 데이터를 1차원 배열로 변환
    img_flat = img.reshape((-1, 3))

    # K-means 클러스터링 모델 생성
    kmeans = KMeans(n_clusters=3)

    # 클러스터링 수행
    kmeans.fit(img_flat)

    # 클러스터 중심을 얻어오고 이를 정수로 변환
    colors = kmeans.cluster_centers_.astype(int)

    # BGR에서 RGB로 변환
    colors_rgb = colors[:, [2, 1, 0]]

    # 주요 색상 출력
    # print("주요 색상 (RGB):")
    for color in colors_rgb:
        # print(tuple(color))
        main_3_color_list.append(color)

    # 색상 팔레트 생성
    color_palette = np.zeros((50, 120, 3), dtype=np.uint8)
    for i, color in enumerate(colors_rgb):
        color_palette[:, i * 40:(i + 1) * 40, :] = color

    # 이미지 표시
    # plt.imshow(color_palette)
    # plt.axis('off')
    # plt.show()

    return main_3_color_list

##### 이미지 개체 인식

In [2]:
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import cv2
# import tensorflow as tf

def image_sagmentation(img_path):   # 이미지 segmentation으로 개체 분류
    '''
    모델 호출
    '''
    new_model = YOLO('c:/3rd_project/yolov8_pt/230926_a_4_colab_best.pt')   # Mask 모델

    '''
    이미지 적용
    '''
    image = cv2.imread(img_path)
    img = cv2.resize(image, (320, 448))  # (가로, 세로)
    results = list(new_model.predict(source=img,
                                    conf=0.25, show=False, stream=True))
    #                                 예측률0.5초과                      detect할 클래스만 classes=[0, 2, ...] 추가
    #                                                                   없는 경우 생략

    '''
    이미지 출력
    '''
    # res_plotted = results[0].plot()
    # plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    # plt.axis('off')
    # plt.show()

    '''
    개체 수 출력
    '''
    # gpu => cpu
    cls_cpu = results[0].boxes.cls.to('cpu').to(int)
    # cpu => list
    cls_list = cls_cpu.tolist()

    from collections import Counter

    cls_counts = Counter(cls_list)
    # print(cls_counts)

    # 클래스 이름과 개수를 연결하는 딕셔너리 생성
    class_names = {
        new_model.names[class_idx]: str(count)
        for class_idx, count in cls_counts.items()
    }

    # 모든 키-값 쌍을 반복하고 출력
    # for class_name, class_value in class_names.items():
    #     print(f"Class: {class_name}\t\tValue: {class_value}")

    model_classes = new_model.names.items()
    
    return class_names, model_classes

In [4]:
import cv2

img_path = 'c:/3rd_project/data/images/movie/action/164.jpg'
img = cv2.imread(img_path)


# show_origin(img)

# color_hist(img)

brightness_value = brightness(img_path)

main_3_color_list = main_3_color(img)

i_s = image_sagmentation(img_path)


0: 448x320 1 person, 2 Guns, 51.7ms
Speed: 0.0ms preprocess, 51.7ms inference, 5.1ms postprocess per image at shape (1, 3, 448, 320)


In [4]:
print(brightness_value)
print(main_3_color_list)
print(i_s)

0.3308
[array([114, 104,  86]), array([38, 34, 26]), array([218, 215, 208])]
({'person': '3'}, dict_items([(0, 'person'), (1, 'bicycle'), (2, 'car'), (3, 'motorcycle'), (4, 'airplane'), (5, 'bus'), (6, 'train'), (7, 'truck'), (8, 'boat'), (9, 'traffic light'), (10, 'fire hydrant'), (11, 'stop sign'), (12, 'parking meter'), (13, 'bench'), (14, 'bird'), (15, 'cat'), (16, 'dog'), (17, 'horse'), (18, 'sheep'), (19, 'cow'), (20, 'elephant'), (21, 'bear'), (22, 'zebra'), (23, 'giraffe'), (24, 'backpack'), (25, 'umbrella'), (26, 'handbag'), (27, 'tie'), (28, 'suitcase'), (29, 'frisbee'), (30, 'skis'), (31, 'snowboard'), (32, 'sports ball'), (33, 'kite'), (34, 'baseball bat'), (35, 'baseball glove'), (36, 'skateboard'), (37, 'surfboard'), (38, 'tennis racket'), (39, 'bottle'), (40, 'wine glass'), (41, 'cup'), (42, 'fork'), (43, 'knife'), (44, 'spoon'), (45, 'bowl'), (46, 'banana'), (47, 'apple'), (48, 'sandwich'), (49, 'orange'), (50, 'broccoli'), (51, 'carrot'), (52, 'hot dog'), (53, 'pizza')

In [5]:
main_3_color_list[0].tolist()

[114, 104, 86]

In [6]:
# i_s[1]에서 클래스 이름 추출
class_names = [class_name for _, class_name in i_s[1]]

# 클래스 이름 리스트 출력
print(class_names)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'Gun']


In [7]:
columns = class_names.copy()  # class_names 리스트를 복사하여 columns 변수에 할당

# 'name'을 'columns' 리스트의 맨 앞에 추가
columns.insert(0, 'img_name')

# 'genre'을 'columns' 리스트의 끝에 추가
columns.extend(['brightness', 'color_1_r', 'color_1_g', 'color_1_b',
                'color_2_r', 'color_2_g', 'color_2_b',
                'color_3_r','color_3_g', 'color_3_b','genre'])

In [8]:
print(columns)

['img_name', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'Gun', 'brightness', 'color_1_r', 'color_1_g', 'color_1_b', 'color_2_r', 'color_2_g', 'color_2_b', 'color_3_r', 'color_3_g', '

In [9]:
import pandas as pd

df = pd.DataFrame(columns=columns)
df

,img_name,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre


In [10]:
genre = img_path.split('/')[-2]
name = img_path.split('/')[-1]
img_name = f"{genre}_{name}"

print(genre)
print(name)
print(img_name)

action
164.jpg
action_164.jpg


In [11]:
color_1 = main_3_color_list[0].tolist()
color_2 = main_3_color_list[1].tolist()
color_3 = main_3_color_list[2].tolist()

for i in range(1):
    for key, value in i_s[0].items():
        df.loc[i, 'img_name'] = img_name
        df.loc[i, 'brightness'] = brightness_value
        df.loc[i, 'color_1_r'] = color_1[0]
        df.loc[i, 'color_1_g'] = color_1[1]
        df.loc[i, 'color_1_b'] = color_1[2]
        df.loc[i, 'color_2_r'] = color_2[0]
        df.loc[i, 'color_2_g'] = color_2[1]
        df.loc[i, 'color_2_b'] = color_2[2]
        df.loc[i, 'color_3_r'] = color_3[0]
        df.loc[i, 'color_3_g'] = color_3[1]
        df.loc[i, 'color_3_b'] = color_3[2]
        df.loc[i, 'genre'] = genre
        if key in df.columns:
            df.loc[i, key] = value

In [12]:
df = df.fillna(0)
df

,img_name,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre
0,action_164.jpg,3,0,0,0,0,0,0,0,0,...,114,104,86,38,34,26,218,215,208,action


In [13]:
df[df.columns[-13:]]

,toothbrush,Gun,brightness,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre
0,0,0,0.3308,114,104,86,38,34,26,218,215,208,action


액션 폴더 파일 리스트 가져오기

In [14]:
import os

folder_path = 'c:/3rd_project/data/images/movie/action'
files = os.listdir(folder_path)

print(files)
print(len(files))

['0.jpg', '1.jpg', '10.jpg', '100.jpg', '1000.jpg', '1001.jpg', '1002.jpg', '1003.jpg', '1004.jpg', '1005.jpg', '1006.jpg', '1007.jpg', '1008.jpg', '1009.jpg', '101.jpg', '1010.jpg', '1011.jpg', '1012.jpg', '1013.jpg', '1014.jpg', '1015.jpg', '1016.jpg', '1017.jpg', '1018.jpg', '1019.jpg', '102.jpg', '1020.jpg', '1021.jpg', '1022.jpg', '1023.jpg', '1024.jpg', '1025.jpg', '1026.jpg', '1027.jpg', '1028.jpg', '1029.jpg', '103.jpg', '1030.jpg', '1031.jpg', '1032.jpg', '1033.jpg', '1034.jpg', '1035.jpg', '1036.jpg', '1037.jpg', '1038.jpg', '1039.jpg', '104.jpg', '1040.jpg', '1041.jpg', '1042.jpg', '1043.jpg', '1044.jpg', '1045.jpg', '1046.jpg', '1047.jpg', '1048.jpg', '1049.jpg', '105.jpg', '1050.jpg', '1051.jpg', '1052.jpg', '1053.jpg', '1054.jpg', '1055.jpg', '1056.jpg', '1057.jpg', '1058.jpg', '1059.jpg', '106.jpg', '1060.jpg', '1061.jpg', '1062.jpg', '1063.jpg', '1064.jpg', '1065.jpg', '1066.jpg', '1067.jpg', '1068.jpg', '1069.jpg', '107.jpg', '1070.jpg', '1071.jpg', '1072.jpg', '1073.j

In [ ]:
# # i_s[1]에서 클래스 이름 추출
# class_names = [class_name for _, class_name in i_s[1]]

# columns = class_names.copy()  # class_names 리스트를 복사하여 columns 변수에 할당

# # 'name'을 'columns' 리스트의 맨 앞에 추가
# columns.insert(0, 'img_name')

# # 'genre'을 'columns' 리스트의 끝에 추가
# columns.extend(['brightness', 'color_1_r', 'color_1_g', 'color_1_b',
#                 'color_2_r', 'color_2_g', 'color_2_b',
#                 'color_3_r','color_3_g', 'color_3_b','genre'])


# df = pd.DataFrame(columns=columns)
# df

### 통합

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

def show_origin(img):   # 원본 이미지, 크기 출력
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img_rgb)
    plt.axis("off")
    plt.show()

    print(img_rgb.shape)

def color_hist(img):    # 원본 이미지 색 히스토그램 출력
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 그레이 이미지의 0번 채널에 대한 히스토그램 계산
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])

    # RGB 이미지의 각 채널에 대한 히스토그램 계산
    hist_b = cv2.calcHist([img_rgb], [0], None, [256], [0, 256])
    hist_g = cv2.calcHist([img_rgb], [1], None, [256], [0, 256])
    hist_r = cv2.calcHist([img_rgb], [2], None, [256], [0, 256])

    # 히스토그램 시각화
    plt.figure(figsize=(8, 6))
    plt.title('Color Histogram')
    plt.xlabel('Color Value')
    plt.ylabel('Frequency')

    # 그레이 히스토그램 채우기
    plt.fill_between(range(256), hist.flatten(), color='k', alpha=0.4, label='Histogram')

    # RGB 채널 히스토그램 그래프 그리기
    plt.plot(hist_b, color='b', label='Blue Channel')
    plt.plot(hist_g, color='g', label='Green Channel')
    plt.plot(hist_r, color='r', label='Red Channel')

    plt.xlim([0, 256])
    plt.legend()
    plt.show()

def brightness(img_path):   # 원본 이미지 밝기 계산
    image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(image, (500, 750))  # (가로, 세로)

    brightness_value = round(img.mean() / 255 , 4)
    # print(f"이미지 밝기 : {brightness_value}")

    return brightness_value

def main_3_color(img):  # 원본 이미지 대표 색상 3개 rgb 값 반환
    main_3_color_list = []

    # 이미지 데이터를 1차원 배열로 변환
    img_flat = img.reshape((-1, 3))

    # K-means 클러스터링 모델 생성
    kmeans = KMeans(n_clusters=3)

    # 클러스터링 수행
    kmeans.fit(img_flat)

    # 클러스터 중심을 얻어오고 이를 정수로 변환
    colors = kmeans.cluster_centers_.astype(int)

    # BGR에서 RGB로 변환
    colors_rgb = colors[:, [2, 1, 0]]

    # 주요 색상 출력
    # print("주요 색상 (RGB):")
    for color in colors_rgb:
        # print(tuple(color))
        main_3_color_list.append(color)

    # 색상 팔레트 생성
    color_palette = np.zeros((50, 120, 3), dtype=np.uint8)
    for i, color in enumerate(colors_rgb):
        color_palette[:, i * 40:(i + 1) * 40, :] = color

    # 이미지 표시
    # plt.imshow(color_palette)
    # plt.axis('off')
    # plt.show()

    return main_3_color_list

In [2]:
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import cv2
# import tensorflow as tf

def image_sagmentation(img_path):   # 이미지 segmentation으로 개체 분류
    '''
    모델 호출
    '''
    new_model = YOLO('c:/3rd_project/yolov8_pt/230926_a_4_colab_best.pt')   # Mask 모델

    '''
    이미지 적용
    '''
    image = cv2.imread(img_path)
    img = cv2.resize(image, (320, 448))  # (가로, 세로)
    results = list(new_model.predict(source=img,
                                    conf=0.25, show=False, stream=True))
    #                                 예측률0.5초과                      detect할 클래스만 classes=[0, 2, ...] 추가
    #                                                                   없는 경우 생략

    '''
    이미지 출력
    '''
    # res_plotted = results[0].plot()
    # plt.imshow(cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB))
    # cv2.waitKey()
    # cv2.destroyAllWindows()
    # plt.axis('off')
    # plt.show()

    '''
    개체 수 출력
    '''
    # gpu => cpu
    cls_cpu = results[0].boxes.cls.to('cpu').to(int)
    # cpu => list
    cls_list = cls_cpu.tolist()

    from collections import Counter

    cls_counts = Counter(cls_list)
    # print(cls_counts)

    # 클래스 이름과 개수를 연결하는 딕셔너리 생성
    class_names = {
        new_model.names[class_idx]: str(count)
        for class_idx, count in cls_counts.items()
    }

    # 모든 키-값 쌍을 반복하고 출력
    # for class_name, class_value in class_names.items():
    #     print(f"Class: {class_name}\t\tValue: {class_value}")

    model_classes = new_model.names.items()
    
    return class_names, model_classes

In [9]:
# 빈 데이터프레임 만들기
import pandas as pd

columns = ['img_name',
           'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'Gun',
           'brightness', 'color_1_r', 'color_1_g', 'color_1_b', 'color_2_r', 'color_2_g', 'color_2_b', 'color_3_r','color_3_g', 'color_3_b','genre']

df = pd.DataFrame(columns=columns)
df

,img_name,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre


In [11]:
import cv2
from tqdm import tqdm
import os

error_list = []
genre_list = ['action', 'ani', 'comedy', 'drama', 'horror']

folder_path = f"c:/3rd_project/data/images/movie/{genre_list[4]}"
files = os.listdir(folder_path)

for i, file in tqdm(enumerate(files), total=len(files)):
    file_path = f"{folder_path}/{file}"
    try:
        img = cv2.imread(file_path)
        img = cv2.resize(img, (500, 750))  # (가로, 세로)

        brightness_value = brightness(file_path)
        main_3_color_list = main_3_color(img)
        i_s = image_sagmentation(file_path) # 230925_c.pt 모델 가중치로 돌리셈

        color_1 = main_3_color_list[0].tolist()
        color_2 = main_3_color_list[1].tolist()
        color_3 = main_3_color_list[2].tolist()

        genre = file_path.split('/')[-2]
        name = file_path.split('/')[-1]
        img_name = f"{genre}_{name}"

        for key, value in i_s[0].items():
            df.loc[i, 'img_name'] = img_name
            df.loc[i, 'brightness'] = brightness_value
            df.loc[i, 'color_1_r'] = color_1[0]
            df.loc[i, 'color_1_g'] = color_1[1]
            df.loc[i, 'color_1_b'] = color_1[2]
            df.loc[i, 'color_2_r'] = color_2[0]
            df.loc[i, 'color_2_g'] = color_2[1]
            df.loc[i, 'color_2_b'] = color_2[2]
            df.loc[i, 'color_3_r'] = color_3[0]
            df.loc[i, 'color_3_g'] = color_3[1]
            df.loc[i, 'color_3_b'] = color_3[2]
            df.loc[i, 'genre'] = genre
            if key in df.columns:
                df.loc[i, key] = value
    except:
        error_list.append(file_path)

df = df.fillna(0)
df.to_csv(f"c:/3rd_project/data/Train/{genre_list[4]}_movie.csv", index=None)

  0%|          | 0/1493 [00:00<?, ?it/s]


0: 448x320 3 persons, 44.3ms
Speed: 9.0ms preprocess, 44.3ms inference, 29.4ms postprocess per image at shape (1, 3, 448, 320)
  0%|          | 1/1493 [00:02<1:02:23,  2.51s/it]
0: 448x320 1 person, 23.7ms
Speed: 15.0ms preprocess, 23.7ms inference, 5.0ms postprocess per image at shape (1, 3, 448, 320)
  0%|          | 2/1493 [00:05<1:04:39,  2.60s/it]
0: 448x320 (no detections), 15.0ms
Speed: 2.0ms preprocess, 15.0ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 320)
  0%|          | 3/1493 [00:07<59:03,  2.38s/it]  
0: 448x320 (no detections), 47.7ms
Speed: 4.0ms preprocess, 47.7ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 320)
  0%|          | 4/1493 [00:09<56:04,  2.26s/it]
0: 448x320 (no detections), 19.5ms
Speed: 1.0ms preprocess, 19.5ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 320)
  0%|          | 5/1493 [00:12<1:03:36,  2.56s/it]
0: 448x320 2 persons, 39.6ms
Speed: 1.5ms preprocess, 39.6ms inference, 16.0ms postprocess per im

In [12]:
df

,img_name,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre
0,horror_0.jpg,3,0,0,0,0,0,0,0,0,...,200,8,7,242,241,241,69,69,69,horror
1,horror_1.jpg,1,0,0,0,0,0,0,0,0,...,207,217,225,16,29,47,95,120,147,horror
5,horror_1001.jpg,2,0,0,0,0,0,0,0,0,...,181,171,149,12,18,29,79,89,93,horror
11,horror_1007.jpg,1,0,0,0,0,0,0,0,0,...,13,4,3,188,167,162,86,33,27,horror
17,horror_1012.jpg,1,0,0,0,0,0,0,0,0,...,29,17,14,194,178,155,88,67,52,horror
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1480,horror_988.jpg,2,0,0,0,0,0,0,0,0,...,33,27,28,199,173,164,151,66,66,horror
1482,horror_99.jpg,2,0,0,0,0,0,0,0,0,...,105,85,82,178,164,160,35,25,29,horror
1483,horror_990.jpg,0,0,0,0,0,0,0,0,0,...,198,220,232,33,29,36,95,135,170,horror
1484,horror_991.jpg,1,0,0,0,0,0,0,0,0,...,40,7,5,145,33,20,208,146,97,horror


In [9]:
import pandas as pd

df_act = pd.read_csv('c:/3rd_project/data/Train/action_movie.csv')
df_ani = pd.read_csv('c:/3rd_project/data/Train/ani_movie.csv') 
df_com = pd.read_csv('c:/3rd_project/data/Train/comedy_movie.csv')
df_dra = pd.read_csv('c:/3rd_project/data/Train/drama_movie.csv')
df_hor = pd.read_csv('c:/3rd_project/data/Train/horror_movie.csv')

print(f"df_act : {df_act.shape}\ndf_ani : {df_ani.shape}\ndf_com : {df_com.shape}\ndf_dra : {df_dra.shape}\ndf_hor : {df_hor.shape}\n")

df_act : (1092, 93)
df_ani : (836, 93)
df_com : (517, 93)
df_dra : (2217, 93)
df_hor : (821, 93)



In [14]:
dfs = [df_act, df_ani, df_com, df_dra, df_hor]
train_df = pd.concat(dfs, ignore_index=True)
train_df

,img_name,person,bicycle,car,motorcycle,airplane,bus,train,truck,boat,...,color_1_r,color_1_g,color_1_b,color_2_r,color_2_g,color_2_b,color_3_r,color_3_g,color_3_b,genre
0,action_0.jpg,2,0,0,0,0,0,0,0,0,...,35,22,20,188,184,172,117,81,74,action
1,action_1.jpg,1,0,0,0,0,0,0,0,0,...,224,203,173,30,28,34,143,117,97,action
2,action_10.jpg,1,0,0,0,0,0,0,0,0,...,223,211,207,7,5,5,133,108,101,action
3,action_1000.jpg,0,0,0,0,0,0,0,0,0,...,18,17,16,155,138,147,62,44,44,action
4,action_1002.jpg,2,0,0,0,0,0,0,0,0,...,54,39,36,233,213,159,106,116,123,action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5478,horror_988.jpg,2,0,0,0,0,0,0,0,0,...,33,27,28,199,173,164,151,66,66,horror
5479,horror_99.jpg,2,0,0,0,0,0,0,0,0,...,105,85,82,178,164,160,35,25,29,horror
5480,horror_990.jpg,0,0,0,0,0,0,0,0,0,...,198,220,232,33,29,36,95,135,170,horror
5481,horror_991.jpg,1,0,0,0,0,0,0,0,0,...,40,7,5,145,33,20,208,146,97,horror


In [15]:
train_df.to_csv(f"c:/3rd_project/data/Train/train_movie.csv", index=None)